In [ ]:
import numpy as np
import pandas as pd
import pandasql as psql

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import scipy.sparse

from imblearn.over_sampling import SMOTE



# Import DataFrame

In [ ]:
df = pd.read_csv("~/Downloads/bigml_59c28831336c6604c800002a.csv")

In [ ]:
df.head()

# Label Encode Explanatory Data

In [ ]:
le = LabelEncoder()

df['international plan'] = le.fit_transform(df['international plan'])
df['voice mail plan'] = le.fit_transform(df['voice mail plan'])
# df['area code'] = le.fit_transform(df['area code'])
# df['state'] = le.fit_transform(df['state'])

In [ ]:
# ohe = OneHotEncoder()
# state = ohe.fit_transform(df[['state']])
# df = pd.concat((df,pd.DataFrame.sparse.from_spmatrix(state)),axis=1)

# Create Additional Columns to Better Explain the Data

In [ ]:
# df['account months'] = np.ceil(df['account length']/30).astype(int)
df['total minutes'] = df['total day minutes']+df['total eve minutes']+df['total night minutes']+df['total intl minutes']
df['total calls'] = df['total day calls']+df['total eve calls']+df['total night calls']+df['total intl calls']
df['total cost'] = df['total day charge']+df['total eve charge']+df['total night charge']+df['total intl charge']
df['cost per day'] = df['total cost']/df['account length']

# Separate Data into Train/Test Sets and Fit to Random Forest

In [ ]:
rfc = RandomForestClassifier(n_estimators=1000,random_state=42)
X = df.drop(columns = ['churn','phone number','state','area code'])
y = df['churn']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,train_size = .8)
X_test1, X_test_final,y_test1,y_test_final = train_test_split(X_test,y_test,random_state=42,train_size=.5)
sm = SMOTE(random_state=42, sampling_strategy=.9)
# Xsmote, ysmote = sm.fit_sample(X_train, y_train)
# rfc.fit(Xsmote,ysmote)
rfc.fit(X_train,y_train)

# Check Training Data

In [ ]:
print(confusion_matrix(y_train,rfc.predict(X_train)))
print(classification_report(y_train,rfc.predict(X_train)))

# Check on First Set of Test Data

In [ ]:
print(confusion_matrix(y_test1,rfc.predict(X_test1)))
print(classification_report(y_test1,rfc.predict(X_test1)))

# Coefficients 

In [ ]:
sorted(list(zip(rfc.feature_importances_,X_test1.columns,)),reverse=True)

# Double Check on Second Set of Test Data

In [ ]:
print(confusion_matrix(y_test_final,rfc.predict(X_test_final)))
print(classification_report(y_test_final,rfc.predict(X_test_final)))

In [ ]:
df.head()

In [ ]:
gb = GradientBoostingClassifier(learning_rate=.01,n_estimators=9999,subsample=.7,max_depth=2)

In [ ]:
gb.fit(X_train,y_train)

In [ ]:
print(confusion_matrix(y_train,gb.predict(X_train)))
print(classification_report(y_train,gb.predict(X_train)))

In [ ]:
print(confusion_matrix(y_test1,gb.predict(X_test1)))
print(classification_report(y_test1,gb.predict(X_test1)))

In [ ]:
print(confusion_matrix(y_test_final,gb.predict(X_test_final)))
print(classification_report(y_test_final,gb.predict(X_test_final)))

In [ ]:
sorted(list(zip(gb.feature_importances_,X_test1.columns,)),reverse=True)